In [2]:
from src.recom_search.model.beam_node_reverse import ReverseNode
from transformers import AutoTokenizer, AutoModel

import flatten_lattice as fl
import torch
from bert_models import LinearLatticeBert, LinearPOSBert
from encoding_utils import *
import pickle

device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

from mask_utils import *

2022-08-29 05:54:01.220619: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-29 05:54:01.220641: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
bert_tok = AutoTokenizer.from_pretrained("bert-base-cased")
mbart_tok = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
#TODO things to sanity check
# - reversing graphs
# - flattening graphs
# - getting masks from graphs
# - running flattened versions (how do masks / posids fit into the picture)?

In [4]:
s1 = "The Fed raises interest rates"
s2 = "The Fed raises interest him"


In [5]:
def node_from_idx(t1, ind):
    return ReverseNode(None, {'uid':str(t1)+str(ind), 'token_idx':t1, 'token_str':bert_tok.decode(t1), 'prob':0})

# take 2 comparable strings that split at some point, make a simulated lattice 
def create_toy_graph(s1, s2, tok):
    toks1 = tok(s1).input_ids
    toks2 = tok(s2).input_ids
    #print(toks1)
    #print(toks2)
    assert len(toks1) == len(toks2)
    prev = [None, None]
    merged = False
    gdict = {}
    for i in range(len(toks1)):
        
        cur1 = node_from_idx(toks1[i], i)
        if i==0:
            gdict['root'] = cur1
        gdict[cur1.uid] = cur1
        #print(toks1[i] is not toks2[i])
        #print(merged)
        
        if (toks1[i] != toks2[i]) or merged:
            #print("in")
            if prev[1] is None:
                prev[1] = prev[0]
            cur2 = node_from_idx(toks2[i], i)
            prev[1].nextlist.append(cur2)
            prev[1].next_scores.append(cur2.prob)
            prev[1].next_ids.append(cur2.uid)
            prev[1] = cur2
            gdict[cur2.uid] = cur2
            merged = True
        if prev[0]:
            prev[0].nextlist.append(cur1)
            prev[0].next_scores.append(cur1.prob)
            prev[0].next_ids.append(cur1.uid)
        prev[0] = cur1
    return gdict
    

In [6]:
toygraph = create_toy_graph(s1, s2, mbart_tok)

In [7]:
exploded = fl.get_all_possible_candidates(toygraph)

In [8]:
flat_toy = fl.flatten_lattice(toygraph)

In [9]:
p = flat_toy
tlist = fl.get_toklist(p)
decstr = bert_tok.decode(tlist)
print(len(tlist))
#for node in p:
    #print(node['token_idx'], " ", node['pos'])
print(decstr)

6
The Fed raises interest him rates


In [13]:
mask = connect_mat(p)

In [21]:
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AutoConfig
from latmask_bert_models import LatticeBertModel

#TODO fix bug with misaligned 101, 102
class LinearPOSBertV1(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = LatticeBertModel(AutoConfig.from_pretrained('bert-base-cased'))
        self.probe = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.to(device)

    def parameters(self):
        return self.probe.parameters()
  
    def forward(self, sentences, pos_ids=None, attmasks=None):
        with torch.no_grad(): # no training of BERT parameters
            if pos_ids==None:
                word_rep, sentence_rep = self.bert(sentences, return_dict=False)
            else:
                print('YO')
                word_rep, sentence_rep = self.bert(sentences, position_ids=pos_ids, encoder_attention_mask=attmasks, attention_mask=attmasks, return_dict=False)
        return self.probe(word_rep)
    
def prepare_dataset(resset):
    x = []
    y = []
    for res in resset:
        curinps = []
        for r in res:
            try:
                toktmp = torch.tensor(bert_tok(clean_expanded(r)).input_ids)
                #print(toktmp.shape)
                if float(toktmp.shape[0])<MAX_LEN:
                    toktmp = torch.cat([toktmp, torch.zeros(MAX_LEN-toktmp.shape[0])])
                else:
                    toktmp = toktmp[:MAX_LEN]
                curinps.append(toktmp)
            except:
                print("weird error happened") 
        print(len(curinps))
        curouts = []
        tinp = torch.stack(curinps).long().to(device)
        print(tinp.shape)
        y.append(posbmodel(tinp))
        x.append(tinp)
        
        #print("error somewhere")
    return x, y

def fix_posids(pids):
    cop = pids
    for p in cop:
        for i in range(0, len(p)):
            p[i] = i
    return cop

In [12]:
import json
with open('./a3-distrib/lab_vocab.json') as json_file:
    labels = json.load(json_file)
posbmodel = LinearPOSBertV1(len(list(labels.keys())))
posbmodel.load_state_dict(torch.load("./a3-distrib/ckpt/posbert.pth"))
posbmodel.eval()
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated("cuda:2"))

433909248


In [23]:
from encoding_utils import *

# get gold labels for the exploded set
dsetx, dsety = prepare_dataset([exploded])
print(len(dsetx))
#print(torch.cuda.memory_allocated("cuda:2"))
assert len(dsetx)==1
sents, posids = create_inputs([flat_toy])
latposylabels, tmaps = lattice_pos_goldlabels(dsetx, dsety, sents)

#latposylabels = tmap_pos_goldlabels(tmaps, sents)

# get generated labels
pred2 = posbmodel(sents.to(device), fix_posids(posids).to(device), torch.stack([mask]).to(device))

# check to see if there are errors

2
torch.Size([2, 500])
LatticeBertModel ATT MASK SHAPE
LatticeBertEncoder ATT MASK SHAPE
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
1
2
0
YO
LatticeBertModel ATT MASK SHAPE
LatticeBertEncoder ATT MASK SHAPE
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
None
LatticeBertAttention ATT MASK SHAPE
ATT MASK SHAPE
N

In [26]:
pred2[0].shape

torch.Size([500, 44])

In [48]:
def check_accuracy(setpred, setlabels):
    cor = 0
    tot = 0
    for i in range(0, len(setpred)):
        ex = setpred[i]
        for j in range(0, len(ex)):
            if sum(setlabels[i][j])==0:
                continue
            elif torch.argmax(ex[j])==0:
                continue
            tot+=1
            if torch.argmax(ex[j])==torch.argmax(setlabels[i][j]):
                cor+=1
    return cor/tot

check_accuracy(pred2, latposylabels)

0.8571428571428571

In [29]:
lablist = [k for k in labels.keys()]

In [32]:
def show_labels (pred):
    res = []
    for p in pred:
        res.append(lablist[torch.argmax(p)])
    return res

print(show_labels(pred2[0])[:8])
print(show_labels(latposylabels[0])[:8])
"The fed raises interest rates him."

['<pad>', 'DT', 'NNP', 'VBZ', 'NN', 'RB', 'NNS', '.']
['<pad>', 'DT', 'NNP', 'VBZ', 'NN', 'PRP', 'NNS', '.']


In [ ]:
indivlabs = posbmodel(dsetx[0])

In [47]:
print(show_labels(indivlabs[0])[:8])

['<pad>', 'DT', 'NNP', 'VBZ', 'NN', 'PRP', '.', '<pad>']
